In [3]:
import pandas as pd
from pathlib import Path

In [13]:
ROOT = Path("~/vesuvius-scroll-detection/data/raw/vesuvius").expanduser()

df = pd.read_csv(ROOT / "train.csv")

print(df.head())

         id  scroll_id
0   1407735      26002
1   2290837      34117
2   3320274      35360
3   8862040      34117
4  11460685      34117


In [14]:
print("\nColumns:", list(df.columns))
print("\nNumber of rows:", len(df))


Columns: ['id', 'scroll_id']

Number of rows: 806
